<a href="https://colab.research.google.com/github/AkshitAggarwal/TSAI_EVA5B2_Phase1/blob/main/Session_04/Assignment_Session04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils 
import torch.autograd
import torch.optim as optim
import torchvision.transforms as transforms

#Train Dataset and Train Dataloader

In [2]:
train_data = torchvision.datasets.EMNIST(
    root ='\.bin',
    download = True,
    train = True,
    split = 'byclass',
    transform = transforms.ToTensor()
)

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 32,
    shuffle = True,
)

Extracting \.bin/EMNIST/raw/emnist.zip to \.bin/EMNIST/raw
Processing byclass


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Processing bymerge
Processing balanced
Processing letters
Processing digits
Processing mnist
Done!


In [36]:
print(len(train_data), len(train_loader)) #Total no. of Training images and Training batches(Each batch has 32 images)

697932 21811


In [37]:
print(train_data.train_labels.bincount()) #Individual counts of all classes. Total classes: 62

tensor([34585, 38374, 34203, 35143, 33535, 31416, 34232, 35754, 33946, 33847,
         6407,  3878, 10094,  4562,  4934,  9182,  2517,  3152, 11946,  3762,
         2468,  5076,  9002,  8237, 24983,  8347,  2605,  5073, 20764,  9820,
        12602,  4637,  4695,  2771,  4743,  2701, 10033,  5159,  2854, 10177,
        24631,  2561,  3687,  8738,  2725,  1896,  2491, 15318,  2645, 11418,
         2749,  2448,  2994, 14105,  2699, 18262,  2830,  2910,  2697,  2822,
         2365,  2725])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [5]:
type(train_data)

torchvision.datasets.mnist.EMNIST

In [6]:
batch_iterator = next(iter(train_loader))
type(batch_iterator)

list

In [7]:
images, labels = batch_iterator
print(images.shape, labels.shape)

torch.Size([32, 1, 28, 28]) torch.Size([32])


In [10]:
sample = next(iter(train_data))
print(type(sample))
image, label = sample
print(image.shape, label)

<class 'tuple'>
torch.Size([1, 28, 28]) 35


#Network Architecture

In [20]:
class neural_net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3) #insize= 1x28x28; outsize= 10x26x26; RF= 3
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3) #insize= 10x26x26; outsize= 10x24x24; RF= 5
    self.maxp1 = nn.MaxPool2d(kernel_size = 2, stride = 2) #insize= 10x24x24; outsize= 10x12x12; RF= 10
    self.conv3 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3) #insize= 10x12x12; outsize= 20x10x10; RF= 12
    self.conv4 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=3) #insize= 20x10x10; outsize= 20x8x8; RF= 14
    self.maxp2 = nn.MaxPool2d(kernel_size = 2, stride = 2) #insize= 20x8x8; outsize= 20x4x4; RF= 28
    self.conv5 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3) #insize= 20x4x4; outsize= 30x2x2; RF= 30
    self.conv6 = nn.Conv2d(in_channels=30, out_channels=1, kernel_size=1) #insize = 30x2x2; outsize= 1x2x2; RF= 30
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) 
  
  def forward(self, t):
    t = F.relu(self.maxp1(self.conv2(self.conv1(t)))) #Convolution Block 1
    t = F.relu(self.maxp2(self.conv4(self.conv3(t)))) #Convolution Block 2
    t = self.avgpool(self.conv6(self.conv5(t))) #Convolution Block 3
    return t         


In [21]:
network = neural_net()
print(network)

neural_net(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
  (maxp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
  (maxp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(30, 1, kernel_size=(1, 1), stride=(1, 1))
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
)


In [40]:
pred = network(image.unsqueeze(dim=0)) #Image is a rank-3 tensor, but our model expects a rank-4 'batch-like' tensor.
print(pred.squeeze().shape) 

torch.Size([])


In [41]:
pred.eq(label)

tensor([[[[False]]]])

In [39]:
# for param in network.parameters():
  # print(param.shape)
for name, param in network.named_parameters():
  print(name, '\t\t', param.shape)

conv1.weight 		 torch.Size([10, 1, 3, 3])
conv1.bias 		 torch.Size([10])
conv2.weight 		 torch.Size([10, 10, 3, 3])
conv2.bias 		 torch.Size([10])
conv3.weight 		 torch.Size([20, 10, 3, 3])
conv3.bias 		 torch.Size([20])
conv4.weight 		 torch.Size([20, 20, 3, 3])
conv4.bias 		 torch.Size([20])
conv5.weight 		 torch.Size([30, 20, 3, 3])
conv5.bias 		 torch.Size([30])
conv6.weight 		 torch.Size([1, 30, 1, 1])
conv6.bias 		 torch.Size([1])
